In [146]:
import sys
sys.path.append('../')
import Read_data_and_Write_results as rw
import numpy as np
import visualization_helper as vh
from nltk.stem import PorterStemmer as stemmer

In [147]:
_, words_lsts, bios_lsts, _, _, _ = rw.read_data("../datasets/train.txt")

In [148]:
vocab_words = vh.get_occurrences_word(words_lsts)

In [149]:
words = [key for key in vocab_words.keys()]

In [150]:
def build_matrix(number_of_sentences, number_of_words):
    word_matrix = np.zeros((number_of_sentences, number_of_words))
    return word_matrix

In [151]:
def represent_sentences_as_matrix(matrix, words_lsts, words):
    for i in range(len(words_lsts)):
        for j in range(len(words_lsts[i])):
            mj = words.index( stemmer().stem(words_lsts[i][j].lower()))
            for m in range(9):
                matrix[(i * 9) + m][mj] = 1
    return matrix

In [152]:
def represent_bios_as_matrix(matrix, bios_lsts, words_lsts,  words):
    for i in range(len(words_lsts)):
        for j in range(len(words_lsts[i])):
            d = words.index( stemmer().stem(words_lsts[i][j].lower()))
            for m in range(9):
                if bios_lsts[i][j][m * 2] == "I" or bios_lsts[i][j][m * 2] == "B":
                    matrix[(i * 9) + m][d] = 1
    return matrix

In [153]:
word_matrix = build_matrix(len(words_lsts * 9), len(words))

In [154]:
bio_matrix = build_matrix(len(words_lsts * 9), len(words))

In [155]:
model_matrix = build_matrix(len(words_lsts), len(words))

In [156]:
word_matrix = represent_sentences_as_matrix(word_matrix, words_lsts, words)

In [157]:
bio_matrix = represent_bios_as_matrix(bio_matrix, bios_lsts, words_lsts, words)

In [170]:
def calculate_condetional_probs(w_i, w_j, words_matrix, bios_matrix, model_matrix, words):
    occurrenece_of_wi_and_wj, bold_wi = 0, 0
    occurrenece_of_wi_and_wj = (words_matrix[:, w_i] * words_matrix[:, w_j]).sum()
    y = words_matrix[:, w_i] * words_matrix[:, w_j]
    bold_wi = (y * bios_matrix[:, w_i]).sum()
    model_matrix[w_i][w_j] = (bold_wi / occurrenece_of_wi_and_wj) if occurrenece_of_wi_and_wj else 0
    return model_matrix

In [171]:
model_matrix = calculate_condetional_probs(0, 0, word_matrix, bio_matrix, model_matrix, words)